<a href="https://colab.research.google.com/github/mukundrungta/ReTACRED/blob/main/ReTACRED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install colabcode

In [ ]:
from colabcode import ColabCode
ColabCode()

Code Server can be accessed on: NgrokTunnel: "https://38ad-34-122-52-112.ngrok.io" -> "http://localhost:10000"
[2021-09-20T16:18:14.990Z] info  code-server 3.10.2 387b12ef4ca404ffd39d84834e1f0776e9e3c005
[2021-09-20T16:18:14.993Z] info  Using user-data-dir ~/.local/share/code-server
[2021-09-20T16:18:15.011Z] info  Using config file ~/.config/code-server/config.yaml
[2021-09-20T16:18:15.011Z] info  HTTP server listening on http://127.0.0.1:10000 
[2021-09-20T16:18:15.012Z] info    - Authentication is disabled 
[2021-09-20T16:18:15.012Z] info    - Not serving HTTPS 


In [ ]:
import json
import os
from copy import deepcopy
from tqdm import tqdm

def load_json(path):
    return json.load(open(path, 'r'))

def save_json(data, path):
    return json.dump(data, open(path, 'w'))

def apply_patch_on_split(split_data, split_patch):
    patched_split = []
    for instance in tqdm(split_data):
        instance_id = instance['id']
        if instance_id in split_patch.keys():
            new_instance = deepcopy(instance)
            new_instance['relation'] = split_patch[instance_id]
            patched_split.append(new_instance)
    return patched_split

def apply_patches(split_dir, patch_dir, split_names=['train', 'dev', 'test']):
    patches = {name: None for name in split_names}
    for name in split_names:
        # Set paths
        split_path = os.path.join(split_dir, name + '.json')
        patch_path = os.path.join(patch_dir, name + '_id2label.json')
        # Load data
        split_data = load_json(split_path)
        patch_id2label = load_json(patch_path)
        patch = apply_patch_on_split(split_data, patch_id2label)
        patches[name] = patch
    return patches

def save_patches(patch2data, save_dir):
    for patch_name, data in patch2data.items():
        save_path = os.path.join(save_dir, patch_name + '.json')
        save_json(data, save_path)

In [ ]:
if __name__ == '__main__':
    tacred_dir = 'drive/MyDrive/ColabNotebooks/dataset/tacred' # Directory where TACRED is stored
    patch_dir = 'drive/MyDrive/ColabNotebooks/dataset/id2label' # Directory where patches are located
    # patch_dir = os.getcwd() # Directory where patches are located
    save_dir = 'drive/MyDrive/ColabNotebooks/dataset/retacred' # Directory where patched data should be saved
    os.makedirs(save_dir, exist_ok=True)
    # Apply patches on data splits
    patch2data = apply_patches(split_dir=tacred_dir, patch_dir=patch_dir)
    # Save patched data to desired directory
    save_patches(patch2data=patch2data, save_dir=save_dir)

100%|██████████| 15509/15509 [00:02<00:00, 6080.91it/s]


In [3]:
!sh drive/MyDrive/ColabNotebooks/RE_improved_baseline/scripts/run_bert_base_retacred.sh

wandb: Currently logged in as: mukundrungta (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.12.2
wandb: Syncing run bert-base
wandb: ⭐️ View project at https://wandb.ai/mukundrungta/RE_baseline
wandb: 🚀 View run at https://wandb.ai/mukundrungta/RE_baseline/runs/tz7776r6
wandb: Run data is saved locally in /content/wandb/run-20210920_160758-tz7776r6
wandb: Run `wandb offline` to turn off syncing.

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSeque

'1.9.0+cu102'

In [1]:
!pip install wandb
!pip install ujson
!pip install transformers

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 97 kB 6.0 MB/s 
     |████████████████████████████████| 133 kB 42.8 MB/s 
     |████████████████████████████████| 180 kB 46.8 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=7abc46da333923667b5427f027a806ec9b3c88a3c869d977159a57d180a2fe2d
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=f429082c296254eb7a2730b1ed76cbc7702952ea514b7db057285ea4d8649a9c
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
     |████████████████████████████████| 178 kB 5.2 MB/s 
     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |███████████████████████

In [ ]:
import argparse
import os
import json
import numpy as np
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoConfig, AutoTokenizer
from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from drive.MyDrive.ColabNotebooks.RE_improved_baseline.utils import set_seed, collate_fn
from drive.MyDrive.ColabNotebooks.RE_improved_baseline.prepro import RETACREDProcessor
from drive.MyDrive.ColabNotebooks.RE_improved_baseline.evaluation import get_f1
from drive.MyDrive.ColabNotebooks.RE_improved_baseline.model import REModel
from torch.cuda.amp import GradScaler
import wandb

In [ ]:
def train(args, model, train_features, benchmarks):
    train_dataloader = DataLoader(train_features, batch_size=args.train_batch_size, shuffle=True, collate_fn=collate_fn, drop_last=True)
    total_steps = int(len(train_dataloader) * args.num_train_epochs // args.gradient_accumulation_steps)
    warmup_steps = int(total_steps * args.warmup_ratio)

    scaler = GradScaler()
    optimizer = AdamW(model.parameters(), lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)
    print('Total steps: {}'.format(total_steps))
    print('Warmup steps: {}'.format(warmup_steps))

    num_steps = 0
    for epoch in range(int(args.num_train_epochs)):
        model.zero_grad()
        for step, batch in enumerate(tqdm(train_dataloader)):
            model.train()
            inputs = {'input_ids': batch[0].to(args.device),
                      'attention_mask': batch[1].to(args.device),
                      'labels': batch[2].to(args.device),
                      'ss': batch[3].to(args.device),
                      'os': batch[4].to(args.device),
                      }
            outputs = model(**inputs)
            loss = outputs[0] / args.gradient_accumulation_steps
            scaler.scale(loss).backward()
            if step % args.gradient_accumulation_steps == 0:
                num_steps += 1
                if args.max_grad_norm > 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                scaler.step(optimizer)
                scaler.update()
                scheduler.step()
                model.zero_grad()
                wandb.log({'loss': loss.item()}, step=num_steps)

            if (num_steps % args.evaluation_steps == 0 and step % args.gradient_accumulation_steps == 0):
                for tag, features in benchmarks:
                    f1, output = evaluate(args, model, features, tag=tag, steps=step)
                    wandb.log(output, step=num_steps)

    for tag, features in benchmarks:
        f1, output = evaluate(args, model, features, tag=tag)
        wandb.log(output, step=num_steps)


def evaluate(args, model, features, tag='dev',steps=0):
    dataloader = DataLoader(features, batch_size=args.test_batch_size, collate_fn=collate_fn, drop_last=False)
    keys, preds = [], []
    pred_list = []
    for i_b, batch in enumerate(dataloader):
        model.eval()

        inputs = {'input_ids': batch[0].to(args.device),
                  'attention_mask': batch[1].to(args.device),
                  'ss': batch[3].to(args.device),
                  'os': batch[4].to(args.device),
                  }
        keys += batch[2].tolist()
        with torch.no_grad():
            logit = model(**inputs)[0]
            pred = torch.argmax(logit, dim=-1)
        preds += pred.tolist()
        pred_list.append({'input_ids':imput['input_ids'],'keys':keys,'preds':preds})
    keys = np.array(keys, dtype=np.int64)
    preds = np.array(preds, dtype=np.int64)
    _, _, max_f1 = get_f1(keys, preds)
    data  = json.dumps(pred_list)
    with open('drive/MyDrive/ColabNotebooks/dataset/retacred/predBert'+tag+'_'+steps+'pred.json', 'w') as f:
        json.dump(data, f)
    output = {
        tag + "_f1": max_f1 * 100,
    }
    print(output)
    return max_f1, output

In [ ]:

def main():
    parser = argparse.ArgumentParser()

    parser.add_argument("--data_dir", default="./data/retacred", type=str)
    parser.add_argument("--model_name_or_path", default="roberta-large", type=str)
    parser.add_argument("--input_format", default="typed_entity_marker_punct", type=str,
                        help="in [entity_mask, entity_marker, entity_marker_punct, typed_entity_marker, typed_entity_marker_punct]")

    parser.add_argument("--config_name", default="", type=str,
                        help="Pretrained config name or path if not the same as model_name")
    parser.add_argument("--tokenizer_name", default="", type=str,
                        help="Pretrained tokenizer name or path if not the same as model_name")
    parser.add_argument("--max_seq_length", default=512, type=int,
                        help="The maximum total input sequence length after tokenization. Sequences longer "
                             "than this will be truncated.")
    
    parser.add_argument("--train_batch_size", default=32, type=int,
                        help="Batch size for training.")
    parser.add_argument("--test_batch_size", default=32, type=int,
                        help="Batch size for testing.")
    parser.add_argument("--learning_rate", default=3e-5, type=float,
                        help="The initial learning rate for Adam.")
    parser.add_argument("--gradient_accumulation_steps", default=2, type=int,
                        help="Number of updates steps to accumulate the gradients for, before performing a backward/update pass.")
    parser.add_argument("--adam_epsilon", default=1e-6, type=float,
                        help="Epsilon for Adam optimizer.")
    parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
    parser.add_argument("--warmup_ratio", default=0.1, type=float,
                        help="Warm up ratio for Adam.")
    parser.add_argument("--num_train_epochs", default=5.0, type=float,
                        help="Total number of training epochs to perform.")
    parser.add_argument("--seed", type=int, default=42,
                        help="random seed for initialization")
    parser.add_argument("--num_class", type=int, default=40)
    parser.add_argument("--evaluation_steps", type=int, default=500,
                        help="Number of steps to evaluate the model")

    parser.add_argument("--dropout_prob", type=float, default=0.1)
    parser.add_argument("--project_name", type=str, default="RE_baseline")
    parser.add_argument("--run_name", type=str, default="re-tacred")

    args = parser.parse_args()
    wandb.init(project=args.project_name, name=args.run_name)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    args.n_gpu = torch.cuda.device_count()
    args.device = device
    if args.seed > 0:
        set_seed(args)

    config = AutoConfig.from_pretrained(
        args.config_name if args.config_name else args.model_name_or_path,
        num_labels=args.num_class,
    )
    config.gradient_checkpointing = True
    tokenizer = AutoTokenizer.from_pretrained(
        args.tokenizer_name if args.tokenizer_name else args.model_name_or_path,
    )

    model = REModel(args, config)
    model.to(0)

    train_file = os.path.join(args.data_dir, "train.json")
    dev_file = os.path.join(args.data_dir, "dev.json")
    test_file = os.path.join(args.data_dir, "test.json")

    processor = RETACREDProcessor(args, tokenizer)
    train_features = processor.read(train_file)
    dev_features = processor.read(dev_file)
    test_features = processor.read(test_file)

    if len(processor.new_tokens) > 0:
        model.encoder.resize_token_embeddings(len(tokenizer))

    benchmarks = (
        ("dev", dev_features),
        ("test", test_features),
    )

    # train(args, model, train_features, benchmarks)


if __name__ == "__main__":
    main()

usage: ipykernel_launcher.py [-h] [--data_dir DATA_DIR]
                             [--model_name_or_path MODEL_NAME_OR_PATH]
                             [--input_format INPUT_FORMAT]
                             [--config_name CONFIG_NAME]
                             [--tokenizer_name TOKENIZER_NAME]
                             [--max_seq_length MAX_SEQ_LENGTH]
                             [--train_batch_size TRAIN_BATCH_SIZE]
                             [--test_batch_size TEST_BATCH_SIZE]
                             [--learning_rate LEARNING_RATE]
                             [--gradient_accumulation_steps GRADIENT_ACCUMULATION_STEPS]
                             [--adam_epsilon ADAM_EPSILON]
                             [--max_grad_norm MAX_GRAD_NORM]
                             [--warmup_ratio WARMUP_RATIO]
                             [--num_train_epochs NUM_TRAIN_EPOCHS]
                             [--seed SEED] [--num_class NUM_CLASS]
                             [--eval

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
!python train_retacred.py --model_name_or_path bert-base-cased --input_format typed_entity_marker --seed 78 --train_batch_size 64 --test_batch_size 64 --learning_rate 5e-5 --gradient_accumulation_steps 1 --run_name bert-base;

python3: can't open file 'main': [Errno 2] No such file or directory


In [ ]:
torch._C._cuda_init()